# Sequence-to-Sequence Learning
---
This notebook illustrates the detailed steps of english-german translation model as a *sequence to sequence* problem.

***Reference***: [TensorFlow in Action](https://www.google.de/books/edition/TensorFlow_in_Action/JYyKEAAAQBAJ?hl=en&gbpv=0).

In [1]:
import os
import requests
import zipfile

Download [deu-eng dataset](http://www.manythings.org/anki/deu-eng.zip) manually and locate it in the ```\data``` folder.

In [2]:
# Make sure the zip file has been downloaded
if not os.path.exists(os.path.join('data','deu-eng.zip')):
    raise FileNotFoundError(
    "Uh oh! Did you download the deu-eng.zip from http:/ /www.manythings.org/anki/deu-eng.zip manually and place it in the /data folder?")
else:
    if not os.path.exists(os.path.join('data', 'deu.txt')):
        with zipfile.ZipFile(os.path.join('data','deu-eng.zip'), 'r') as zip_ref:
            zip_ref.extractall('data')
    else:
        print("The extracted data already exists")

The extracted data already exists


In [3]:
import pandas as pd

# Read the csv file
df = pd.read_csv(os.path.join('data', 'deu.txt'), delimiter='\t', header=None)

# Set column names
df.columns = ["EN", "DE", "Attribution"]
df = df[["EN", "DE"]]

In [4]:
print('df.shape = {}'.format(df.shape))

df.shape = (277891, 2)


In [5]:
clean_inds = [i for i in range(len(df)) if b"\xc2" not in df.iloc[i]["DE"].encode("utf-8")]
df = df.iloc[clean_inds]

df.head()

,EN,DE
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!
3,Run!,Lauf!
4,Run.,Lauf!


In [6]:
n_samples = 50000
random_seed= 4321
df = df.sample(n=n_samples, random_state=random_seed)

In [7]:
start_token = 'sos'
end_token = 'eos'
df["DE"] = start_token + ' ' + df["DE"] + ' ' + end_token

In [8]:
# Randomly sample 10% examples from the total 50000 randomly
test_df = df.sample(n=int(n_samples/10), random_state=random_seed)

# Randomly sample 10% examples from the remaining randomly
valid_df = df.loc[~df.index.isin(test_df.index)].sample(n=int(n_samples/10), random_state=random_seed)

# Assign the rest to training data
train_df = df.loc[~(df.index.isin(test_df.index) | df.index.isin(valid_df.index))]

### Analyzing the vocabulary size

In [9]:
from collections import Counter

en_words = train_df["EN"].str.split().sum()
de_words = train_df["DE"].str.split().sum()
n=10

def get_vocabulary_size_greater_than(words, n, verbose=True):
    """
    Get the vocabulary size above a certain threshold
    """
    counter = Counter(words)
    
    freq_df = pd.Series(
    list(counter.values()),
    index=list(counter.keys())
    ).sort_values(ascending=False)
    
    if verbose:
        print(freq_df.head(n=10))
    n_vocab = (freq_df>=n).sum()
    if verbose:
        print("\nVocabulary size (>={} frequent): {}".format(n, n_vocab))
    return n_vocab


print("English corpus")
print('='*50)
en_vocab = get_vocabulary_size_greater_than(en_words, n)

print("\nGerman corpus")
print('='*50)
de_vocab = get_vocabulary_size_greater_than(de_words, n)

English corpus
Tom    9228
to     8700
I      8620
the    6766
you    6136
a      5741
is     4141
in     2639
of     2470
was    2380
dtype: int64

Vocabulary size (>=10 frequent): 2218

German corpus
sos      40000
eos      40000
Tom       9713
Ich       7964
ist       4735
nicht     4616
zu        3606
Sie       3441
du        3132
das       2987
dtype: int64

Vocabulary size (>=10 frequent): 2483


### Analyzing the sequence length

In [10]:
def print_sequence_length(str_ser):
    """
    Print the summary stats of the sequence length
    """
    seq_length_ser = str_ser.str.split(' ').str.len()
    print("\nSome summary statistics")
    print("Median length: {}\n".format(seq_length_ser.median()))
    print(seq_length_ser.describe())
    
    print("\nComputing the statistics between the 1% and 99% quantiles (to ignore outliers)")
    
    p_01 = seq_length_ser.quantile(0.01)
    p_99 = seq_length_ser.quantile(0.99)
    
    print(seq_length_ser[
    (seq_length_ser >= p_01) & (seq_length_ser < p_99)
    ].describe())

In [11]:
print("English corpus")
print('='*50)
print_sequence_length(train_df["EN"])
print("\nGerman corpus")
print('='*50)
print_sequence_length(train_df["DE"])

English corpus

Some summary statistics
Median length: 6.0

count    40000.000000
mean         6.294025
std          2.542850
min          1.000000
25%          5.000000
50%          6.000000
75%          8.000000
max         44.000000
Name: EN, dtype: float64

Computing the statistics between the 1% and 99% quantiles (to ignore outliers)
count    39584.000000
mean         6.184671
std          2.284073
min          2.000000
25%          5.000000
50%          6.000000
75%          7.000000
max         14.000000
Name: EN, dtype: float64

German corpus

Some summary statistics
Median length: 8.0

count    40000.000000
mean         8.332250
std          2.536094
min          3.000000
25%          7.000000
50%          8.000000
75%         10.000000
max         52.000000
Name: DE, dtype: float64

Computing the statistics between the 1% and 99% quantiles (to ignore outliers)
count    39227.000000
mean         8.253116
std          2.231582
min          5.000000
25%          7.000000
50%    

In [13]:
print("EN vocabulary size: {}".format(en_vocab))
print("DE vocabulary size: {}".format(de_vocab))
print("\n")
# Define sequence lengths with some extra space for longer sequences
en_seq_length = 19
de_seq_length = 21
print("EN max sequence length: {}".format(en_seq_length))
print("DE max sequence length: {}".format(de_seq_length))

EN vocabulary size: 2218
DE vocabulary size: 2483


EN max sequence length: 19
DE max sequence length: 21


### Writing an English-German seq2seq Machine Translator
---

Machine translation involves transforming text from one language to another. In this notebook, I focus on creating an *English-to-German* translator using a *sequence-to-sequence* (*seq2seq*) deep learning model.

**Seq2Seq Model Architecture**. The seq2seq model consists of two primary components:
- *Encoder:*
    - Processes the input (English) text and generates a fixed-length context vector, also known as a "*[thought vector](https://wiki.pathmind.com/thought-vectors)*."
    - Encodes the input sequence into a hidden representation that summarizes its semantic and syntactic content.
- *Decoder:*
  - Takes the context vector produced by the encoder as input.
  - Decodes it to produce the output sequence (German text).

Both the encoder and decoder are *recurrent neural networks* (*RNNs*), making them suitable for handling sequential data of arbitrary lengths.

**Key Challenges in Seq2Seq Learning**:
- *Variable Lengths*: The input and output sequences often differ in length. For example, the number of words in a translation might be fewer or greater than the source text.
- *Mapping Arbitrary Sequences*: The model must map an input sequence of arbitrary length to an output sequence of arbitrary length while preserving contextual meaning.

![encoder-decoder-machine-translation](plots/enc-dec.svg)

The *encoder* in our sequence-to-sequence model is built using a *Gated Recurrent Unit* (*GRU*), a type of *RNN*. Its role is to process the input sequence and produce a fixed-size output that summarizes the sequence's information. The *GRU* processes each element of the input sequence step by step, updating its hidden state based on the current input and the previous hidden state. After processing the entire sequence, the final hidden state of the *GRU* serves as the context vector, which encapsulates the semantic and syntactic information of the input.

The *decoder*, which is also based on a *GRU*, further processes the context vector to generate the target sequence. In addition to the *GRU*, the *decoder* incorporates *Dense layers*, which play a critical role in producing the final output. The Dense layers map the *GRU's* outputs to the target vocabulary, generating a probability distribution over possible words at each time step. A key feature of the *decoder* is that the weights of the Dense layers are shared across time steps. This means that, similar to how the *GRU* updates and reuses the same weights for each input in the sequence, the Dense layers also reuse their weights for predicting each word in the output sequence. This approach ensures consistency and efficiency in processing sequential data.

![encoder-decoder-machine-translation](plots/gru.svg)


### The TextVectorization layer
---

The TextVectorization layer takes in a string, tokenizes it, and converts the tokens to *IDs* by means of a vocabulary (or dictionary) lookup. It takes a list of strings (or an array of strings) as the input, where each string can be a *word/phrase/sentence* (and so on). Then it learns the vocabulary from that corpus. Finally, the layer can be used to convert a list of strings to a tensor that contains a sequence of token IDs for each string in the list provided.

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

en_vectorize_layer = TextVectorization(
max_tokens=en_vocab,
output_mode='int',
output_sequence_length=None
)